## Assertions
Assertions in Python are a debugging tool that checks if a condition is true. If the condition is true, the program continues to execute. If it's false, an `AssertionError` is raised, halting the program (unless handled with a `try...except` block).  They are primarily used to catch internal inconsistencies or assumptions in your code during development and testing.  They are *not* intended for handling expected errors like invalid user input.

Here's a breakdown with examples:

**Basic Syntax:**

In [ ]:
assert condition, optional_message

* `condition`: An expression that evaluates to True or False.
* `optional_message`: A string that is displayed if the assertion fails.  This is helpful for providing context.

**Examples:**

1. **Simple Assertion:**

In [ ]:
x = 5
assert x > 0  # This will pass because x is indeed greater than 0

y = -2
assert y > 0  # This will raise an AssertionError

In the second case, because `y` is -2 (not greater than 0), the assertion fails, and an `AssertionError` is raised, stopping the program.  You'll typically see a traceback pointing to the line where the assertion failed.

2. **Assertion with a Message:**

In [ ]:
def divide(a, b):
    assert b != 0, "Cannot divide by zero!"  # More informative message
    return a / b

result = divide(10, 2)  # This will work fine
print(result)  # Output: 5.0

result = divide(10, 0)  # This will raise an AssertionError with the specified message

Here, the message "Cannot divide by zero!" is displayed along with the `AssertionError`, making it clearer why the assertion failed.

3. **Assertions for Function Preconditions:**

In [ ]:
def process_data(data):
    assert isinstance(data, list), "Data must be a list"
    assert all(isinstance(item, int) for item in data), "List items must be integers"
    # ... rest of the function ...

This example uses assertions to ensure that the input `data` to the `process_data` function meets certain criteria (it's a list and contains only integers).  This helps catch errors early in the development process.

4. **Assertions for Postconditions (Less Common but Useful):**

In [ ]:
def calculate_discount(price, discount_percentage):
    assert 0 <= discount_percentage <= 100, "Discount percentage must be between 0 and 100"
    discounted_price = price * (1 - discount_percentage / 100)
    assert discounted_price <= price, "Discounted price cannot be greater than original price" # Postcondition
    return discounted_price

## Real world example we need Assertions

Assertions are primarily used for debugging and ensuring that your code behaves as expected under specific conditions. They are *not* intended for handling runtime errors that you expect might occur (use exceptions for that).

**1. Data Validation Before Processing:**

Imagine you're building a function to calculate the average of a list of sensor readings. You expect the readings to be non-negative.

```python
def calculate_average_reading(readings):
    """Calculates the average sensor reading.

    Args:
        readings: A list of sensor readings (should be non-negative).

    Returns:
        The average reading.

    Raises:
        ValueError: If the input list is empty.
    """

    assert isinstance(readings, list), "Readings must be a list"  # Type check
    assert all(reading >= 0 for reading in readings), "All readings must be non-negative" # Data constraint

    if not readings:
        raise ValueError("Readings list is empty") # Regular exception for expected error

    return sum(readings) / len(readings)


# Example usage
readings = [25, 30, 28, 32, 27]
average = calculate_average_reading(readings)
print(f"Average reading: {average}")

# Example with bad data (will cause the program to halt with an AssertionError)
bad_readings = [25, -5, 28, 32, 27]
# average = calculate_average_reading(bad_readings)  # This will trigger the assertion error

empty_readings = []
#average = calculate_average_reading(empty_readings) # This will raise ValueError, which is expected.
```

Here, the assertions ensure:

*   The input `readings` is a list.
*   All readings within the list are non-negative.

If either of these conditions isn't met during development or testing, the assertion will fail, immediately halting execution and giving you a clear indication of where the problem lies.  This is much better than the code continuing with incorrect data and potentially causing more significant problems later.

**2. Checking Postconditions:**

After a function performs a complex operation, assertions can verify the results.

```python
def apply_discount(price, discount_percentage):
    """Applies a discount to a price.

    Args:
        price: The original price.
        discount_percentage: The discount percentage (0-100).

    Returns:
        The discounted price.
    """
    assert 0 <= discount_percentage <= 100, "Discount percentage must be between 0 and 100"

    discounted_price = price * (1 - discount_percentage / 100)

    assert discounted_price <= price, "Discounted price cannot be greater than original price" # Postcondition check
    assert discounted_price >= 0, "Discounted price cannot be negative" # Postcondition check
    return discounted_price

price = 100
discount = 20
final_price = apply_discount(price, discount)
print(f"Final price after discount: {final_price}")

price = 100
discount = 120 # Will cause the program to halt due to assertion error.
#final_price = apply_discount(price, discount)

```

The assertions after the discount calculation confirm that the `discounted_price` is indeed less than or equal to the original price and greater than or equal to zero. This helps catch errors in the discount logic itself.

**3. Internal Invariants:**

Within a class or a complex data structure, assertions can check internal state consistency.

```python
class ShoppingCart:
    def __init__(self):
        self._items = []

    def add_item(self, item, quantity):
        assert quantity > 0, "Quantity must be positive"
        self._items.append((item, quantity))
        self._assert_invariants() # Check invariants after modification

    def remove_item(self, item):
        self._items = [(i, q) for i, q in self._items if i != item]
        self._assert_invariants() # Check invariants after modification

    def _assert_invariants(self):  # Internal method for invariant checking
        total_quantity = sum(q for _, q in self._items)
        assert total_quantity >= 0, "Total quantity cannot be negative" # Invariant
        # Add other relevant invariants here as needed (e.g., no duplicate items)


cart = ShoppingCart()
cart.add_item("Shirt", 2)
cart.add_item("Pants", 1)
cart.remove_item("Shirt")
#cart.add_item("Hat", -1) # Will cause the program to halt due to assertion error.
```

The `_assert_invariants` method checks the cart's internal state (e.g., total quantity) after any modification.  This helps ensure that the cart's data remains consistent.

**Key Points about Assertions:**

*   **Debugging Tool:** Assertions are primarily for development and debugging. They should be used to catch logical errors and assumptions that you make about your code.
*   **Not for Error Handling:** Don't use assertions for handling runtime errors that you anticipate (like file not found, invalid user input, etc.). Use `try...except` blocks for those.
*   **Can Be Disabled:** Assertions can be globally disabled using the `-O` (optimize) flag when running Python: `python -O your_script.py`.  This is why you shouldn't rely on assertions for anything crucial in production code. They are assumed to be inactive in optimized mode.
*   **Clear Messages:** Provide descriptive messages with your assertions to make debugging easier.

By using assertions strategically, you can significantly improve the quality and maintainability of your Python code by catching errors early in the development process. They act as executable documentation of your code's assumptions and help you avoid nasty surprises later on.



## Different from a normal if checking

While they might seem similar at first glance, they serve fundamentally different purposes and have distinct behaviors.

Here's a breakdown of the key differences:

**1. Purpose:**

*   **Assertions:** Primarily for *debugging* and *testing*. They verify assumptions you make about your code's behavior.  They're meant to catch logical errors during development.  Think of them as "sanity checks" for your code's internal state.
*   **`if` statements:** For *control flow* and *error handling*. They handle expected conditions and branching logic in your program.  They're used to deal with situations that might occur during normal program execution.

**2. Behavior:**

*   **Assertions:** If the condition in an `assert` statement is `False`, the program *immediately terminates* with an `AssertionError`. This is a deliberate "fail-fast" approach to pinpoint bugs quickly.
*   **`if` statements:** If the condition in an `if` statement is `False`, the code within the `if` block is simply skipped, and the program continues executing along a different path.

**3. Optimization:**

*   **Assertions:** Can be *disabled* globally using the `-O` flag when running Python (e.g., `python -O my_script.py`). This is a key difference. In optimized mode, assertions are effectively ignored, so they have no performance impact in production.
*   **`if` statements:** Always *executed*, regardless of optimization flags. They are an integral part of your program's logic and cannot be disabled.

**4. Error Handling:**

*   **Assertions:** Not intended for *handling* runtime errors that you expect might occur (e.g., file not found, network issues). Use `try...except` blocks for that.
*   **`if` statements:** Used extensively for error handling in conjunction with `try...except` blocks.

**In summary:**

| Feature         | Assertions                                 | `if` statements                               |
|-----------------|---------------------------------------------|-------------------------------------------------|
| Purpose         | Debugging, testing, verifying assumptions | Control flow, error handling                 |
| Behavior        | Terminates program on failure               | Skips code block on failure                   |
| Optimization    | Can be disabled                           | Always executed                               |
| Error Handling | Not for expected errors                   | Used for handling expected and unexpected errors |

**Example to illustrate the difference:**

```python
def process_data(data):
    assert isinstance(data, list), "Data must be a list"  # Assertion for debugging

    if data is None:  # if statement for expected error/condition
        return "No data provided"  # Handle the case where no data is given

    # ... rest of the data processing logic ...
```

In this example:

*   The `assert` checks if the input is a list during development. If it's not, the program crashes, helping you find a bug in the code that's calling `process_data`.
*   The `if` checks if the data is `None`, which is a condition that you anticipate might happen. It handles this case gracefully by returning a message instead of crashing.

**When to use which:**

*   Use **assertions** for internal checks, assumptions about your code's state, and conditions that *should never* be false.
*   Use **`if` statements** for handling expected conditions, branching logic, and dealing with potential errors during program execution.



**Important Considerations:**

* **Assertions are disabled with `-O` or `-OO`:**  When you run Python with the `-O` (optimize) or `-OO` (optimize further) flags (e.g., `python -O my_program.py`), assertions are effectively removed.  This is done for performance reasons in production code.  Therefore, you should *not* rely on assertions for handling errors that might occur in a production environment. Use proper exception handling (`try...except`) for that.
* **Use Assertions for Internal Logic:** Assertions are best suited for catching bugs in your code's internal logic, not for handling external input errors (like a user entering the wrong data).  For those, use `try...except` blocks.
* **Don't Use Assertions for Critical Errors:** Because assertions can be disabled, you should never use them for anything that could cause a security vulnerability or data corruption.

**In summary:** Assertions are a valuable debugging tool that can help you find and fix bugs in your code early in the development process. They are not a replacement for proper error handling in production code.  Use them to validate assumptions about your code's internal state.